In [1]:
# Translated to .py by Marti Montesinos
# Modified by Peiyu Jing

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

pandas = pd.read_table("swissmetro.dat")
database = db.Database("swissmetro",pandas)

from headers import *

# Specify parameters to estimate
#1: the name of the parameter,
#2: the default value,
#3: a lower bound (or None, if no bound is specified),
#4: an upper bound, (or None, if no bound is specified),
#5: a flag that indicates if the parameter must be estimated (0) or if it keeps its default value.
ASC_CAR	 = Beta('ASC_CAR',0,-1,1,0)
ASC_SBB	 = Beta('ASC_SBB',0,-1,1,1)
ASC_SM	 = Beta('ASC_SM',0,-1,1,0)
B_CAR_COST	 = Beta('B_CAR_COST',0,-1,1,0)
B_HE	 = Beta('B_HE',0,-1,1,0)
B_SM_COST	 = Beta('B_SM_COST',0,-1,1,0)
B_TIME	 = Beta('B_TIME',0,-1,1,0)
B_TRAIN_COST	 = Beta('B_TRAIN_COST',0,-1,1,0)

# Define here arithmetic expressions for name that are not directly 
# available from the data

CAR_AV_SP  = DefineVariable('CAR_AV_SP', CAR_AV    *  (  SP   !=  0  ),database)
SM_COST  = DefineVariable('SM_COST', SM_CO   * (  GA   ==  0  ),database)
TRAIN_AV_SP  = DefineVariable('TRAIN_AV_SP', TRAIN_AV    *  (  SP   !=  0  ),database)
TRAIN_COST  = DefineVariable('TRAIN_COST', TRAIN_CO   * (  GA   ==  0  ),database)
one  = DefineVariable('one',1,database)

#Utilities
Car_SP = ASC_CAR * one + B_TIME * CAR_TT + B_CAR_COST * CAR_CO
SBB_SP = ASC_SBB * one + B_TIME * TRAIN_TT + B_TRAIN_COST * TRAIN_COST + B_HE * TRAIN_HE
SM_SP = ASC_SM * one + B_TIME * SM_TT + B_SM_COST * SM_COST + B_HE * SM_HE
V = {3: Car_SP,1: SBB_SP,2: SM_SP}
av = {3: CAR_AV_SP,1: TRAIN_AV_SP,2: SM_AV}

# Here we use the "biogeme" way for backward compatibility
exclude = (( PURPOSE != 1 ) * (  PURPOSE   !=  3  ) +  ( CHOICE == 0 )) > 0
database.remove(exclude)

# Here we use the "biogeme" way for backward compatibility
exclude = (((  PURPOSE   !=  1  ) * (  PURPOSE   !=  3  )) + (  CHOICE   ==  0  ) + (  AGE == 6  ))>0
database.remove(exclude)

# Estimation of the model
logprob = bioLogLogit(V,av,CHOICE)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "MNL_SM_specific"
results = biogeme.estimate()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)


ASC_CAR=	-0.953
ASC_SM=	-0.425
B_CAR_COST=	-0.0095
B_HE=	-0.00534
B_SM_COST=	-0.0109
B_TIME=	-0.0111
B_TRAIN_COST=	-0.0293
                 Value   Std err     t-test       p-value  Rob. Std err  \
ASC_CAR      -0.953323  0.115100  -8.282570  2.220446e-16      0.134742   
ASC_SM       -0.424677  0.080506  -5.275074  1.327024e-07      0.102659   
B_CAR_COST   -0.009500  0.000903 -10.517683  0.000000e+00      0.001156   
B_HE         -0.005341  0.001006  -5.307740  1.109927e-07      0.001015   
B_SM_COST    -0.010895  0.000526 -20.715243  0.000000e+00      0.000703   
B_TIME       -0.011060  0.000609 -18.151140  0.000000e+00      0.001200   
B_TRAIN_COST -0.029251  0.001126 -25.983397  0.000000e+00      0.001695   

              Rob. t-test  Rob. p-value  
ASC_CAR         -7.075150  1.492806e-12  
ASC_SM          -4.136772  3.522262e-05  
B_CAR_COST      -8.215170  2.220446e-16  
B_HE            -5.261776  1.426705e-07  
B_SM_COST      -15.503337  0.000000e+00  
B_TIME          -9.21299